Este es un ejermplo de keras para la calsificacion de imagenes en este ejemplo hay imagenes de gatos y perro

In [ ]:
import os
import numpy as np
import keras
from keras import layers
from tensorflow import data as tf_data #para usar tensorflow se tiene que utilizar python 3.9 o 3.11 ya que en otras versiones de python no se puede instalar la libreria
import matplotlib.pyplot as plt

num_skipped = 0 
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = b"JFIF" in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Borrar imagenes corruptas
            os.remove(fpath)
print(f"Deleted {num_skipped} images.")

image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")
        data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images
    plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(augmented_images[0]).astype("uint8"))
        plt.axis("off")
        augmented_train_ds = train_ds = train_ds.map( lambda x, y: (data_augmentation(x), y),num_parallel_calls=tf_data.AUTOTUNE) #esta linea originalmente era estaaugmented_train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y)) pero la cambie porque me daba un error

    # Aplica `data_augmentation` a las imágenes de entrenamiento.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)
# La prefetching de muestras en la memoria de la GPU ayuda a maximizar la utilización de la GPU.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)


def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Bloque de entrada
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Dejar a un lado el residuo

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Residual del proyecto
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Agregar de nuevo el residual
        previous_block_activation = x  # Reserve el próximo residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    # Especificamos activation=None para devolver logits.
    outputs = layers.Dense(units, activation=None)(x)
    return keras.Model(inputs, outputs)
model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)



epochs = 25 #esta parte no puede ejucutarla correctamente en mi computadora 

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)
img = keras.utils.load_img("PetImages/Cat/6779.jpg", target_size=image_size)
plt.imshow(img)
img_array = keras.utils.img_to_array(img)
img_array = keras.ops.expand_dims(img_array, 0)  

predictions = model.predict(img_array)
score = float(keras.ops.sigmoid(predictions[0][0]))
print(f"This image is {100 * (1 - score):.2f}% cat and {100 * score:.2f}% dog.")




Cargue los datos: el conjunto de datos Gatos vs Perros:

este comando se debe ejecutar en el cmd o powershell puede no funcionar asi que podemos tomar la url:https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-/kagglecatsanddogs_5340.zip y descargar y descomprimir los datos sin procesar, la carpeta llamada PetImages debe estar en el mismo lugar en que se se encuentra nuestro archivo .py o .ipynb para que puedag funcionar correctamente 

!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-/kagglecatsanddogs_5340.zip
!unzip -q kagglecatsanddogs_5340.zip
!ls
!ls PetImages
